In [1]:
import numpy as np
import pandas as pd


#https://www.kaggle.com/anshuls235/time-series-forecasting-eda-fe-modelling/notebook#highlight%20hl-lexer_wrapper

from tqdm import tqdm

sales_train_eval = pd.read_csv('sales_train_evaluation.csv')
sell_price = pd.read_csv('sell_prices.csv')
calendar = pd.read_csv('calendar.csv')



for d in range(1942,1970):
    col = 'd_' + str(d)
    sales_train_eval[col] = 0



In [32]:
product_id = 'HOBBIES_1_001'
product_data = sales_train_eval[sales_train_eval['item_id'].str.contains(product_id)]
product_sell_price = sell_price[sell_price['item_id'].str.contains(product_id)]


df = pd.melt(
    product_data,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='value').dropna()
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df,
              product_sell_price,
              on=['store_id', 'item_id', 'wm_yr_wk'],
              how='left')

df["event_name_1"] = df["event_name_1"].fillna("no_event")
df["event_name_2"] = df["event_name_2"].fillna("no_event")
df["event_type_1"] = df["event_type_1"].fillna("no_event")
df["event_type_2"] = df["event_type_2"].fillna("no_event")

df["sell_price"] = df["sell_price"].interpolate(method = 'linear')
df["sell_price"] = df["sell_price"].interpolate(method = 'linear', limit_direction = 'backward')


pd.set_option('display.max_columns', None)
df.head()




,id,item_id,dept_id,cat_id,store_id,state_id,d,value,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,9.58
1,HOBBIES_1_001_CA_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,9.58
2,HOBBIES_1_001_CA_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,9.58
3,HOBBIES_1_001_CA_4_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,9.58
4,HOBBIES_1_001_TX_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,Saturday,1,1,2011,no_event,no_event,no_event,no_event,0,0,0,9.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19595,HOBBIES_1_001_TX_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_2,TX,d_1960,0,2016-06-10,11619,Friday,7,6,2016,no_event,no_event,no_event,no_event,1,0,0,8.26
19596,HOBBIES_1_001_TX_3_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_3,TX,d_1960,0,2016-06-10,11619,Friday,7,6,2016,no_event,no_event,no_event,no_event,1,0,0,8.26
19597,HOBBIES_1_001_WI_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_1,WI,d_1960,0,2016-06-10,11619,Friday,7,6,2016,no_event,no_event,no_event,no_event,1,0,0,8.38
19598,HOBBIES_1_001_WI_2_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_2,WI,d_1960,0,2016-06-10,11619,Friday,7,6,2016,no_event,no_event,no_event,no_event,1,0,0,8.38


In [50]:
#Introduce lags
lags = [1,2,3,6,12,24,36]
for lag in lags:
    df['value_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['value'].shift(-lag)

for lag in lags:
    df['value_lag_'+str(lag)] = df['value_lag_'+str(lag)].fillna(0)

df['rolling_value_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['value'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
df['rolling_value_mean'] = df['rolling_value_mean'].fillna(0)

df.head(19400)

,id,item_id,dept_id,cat_id,store_id,state_id,d,value,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,value_lag_1,value_lag_2,value_lag_3,value_lag_6,value_lag_12,value_lag_24,value_lag_36,rolling_value_mean
0,HOBBIES_1_001_CA_1_evaluation,0,0,0,0,0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,24,4,4,2,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,HOBBIES_1_001_CA_2_evaluation,0,0,0,1,0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,24,4,4,2,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,HOBBIES_1_001_CA_3_evaluation,0,0,0,2,0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,24,4,4,2,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,HOBBIES_1_001_CA_4_evaluation,0,0,0,3,0,d_1,0,2011-01-29,11101,Saturday,1,1,2011,24,4,4,2,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,HOBBIES_1_001_TX_1_evaluation,0,0,0,4,1,d_1,0,2011-01-29,11101,Saturday,1,1,2011,24,4,4,2,0,0,0,9.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19395,HOBBIES_1_001_TX_2_evaluation,0,0,0,5,1,d_1940,0,2016-05-21,11617,Saturday,1,5,2016,24,4,4,2,0,0,0,8.26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285645
19396,HOBBIES_1_001_TX_3_evaluation,0,0,0,6,1,d_1940,2,2016-05-21,11617,Saturday,1,5,2016,24,4,4,2,0,0,0,8.26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.428711
19397,HOBBIES_1_001_WI_1_evaluation,0,0,0,7,2,d_1940,1,2016-05-21,11617,Saturday,1,5,2016,24,4,4,2,0,0,0,8.38,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428467
19398,HOBBIES_1_001_WI_2_evaluation,0,0,0,8,2,d_1940,0,2016-05-21,11617,Saturday,1,5,2016,24,4,4,2,0,0,0,8.38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [51]:
from sklearn.preprocessing import LabelEncoder


def label_encoding(train, feature):

    encoder = LabelEncoder()
    encoder.fit(train[feature].values.astype(str))
    train[feature] = encoder.fit_transform(train[feature].values.astype(str))
    
    return train[feature]


df['item_id']  = label_encoding(df,"item_id" )
df['dept_id']  = label_encoding(df,"dept_id" )
df['cat_id'] = label_encoding(df,"cat_id" )
df['store_id']  = label_encoding(df,"store_id" )
df['state_id']  = label_encoding(df,"state_id" )
df['event_name_1']  = label_encoding(df,"event_name_1" )
df['event_name_2']  = label_encoding(df,"event_name_2" )
df['event_type_1']  = label_encoding(df,"event_type_1" )
df['event_type_2']  = label_encoding(df,"event_type_2" )

In [53]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

col = ['item_id', 'store_id', 'state_id',
       'wm_yr_wk', 'event_name_1', 'event_type_1',
       'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
       'sell_price', 'wday','month', 'year', 'value_lag_1', 'value_lag_2', 
       'value_lag_3', 'value_lag_6', 'value_lag_12', 'value_lag_24', 'value_lag_36']

x_train = df[df['date'] <= '2016-03-27']
y_train = x_train['value']
x_val = df[(df['date'] <= '2016-04-24') & (df['date'] > '2016-03-27')]
y_val = x_val['value']
test = df[df['date'] > '2016-04-24']

hyperparameters = {
    'boosting_type': ['gbdt'],
    'metric': ['rmse'],
    'objective': ['regression'],
    'n_jobs': [-1],
    'seed': [236],
    'learning_rate': [0.2],
    'bagging_fraction': [0.75],
    'bagging_freq': [5],
    'colsample_bytree': [0.75]
}

import lightgbm as lgb

train_set = lgb.Dataset(x_train[col], y_train)
val_set = lgb.Dataset(x_val[col], y_train)

model = lgb.train(hyperparameters, train_set, num_boost_round = 100)

val_pred = model.predict(x_val[col])
val_score = mean_squared_error(val_pred, y_val)
print(f' rmse score is {val_score}')






[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001364 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 405
[LightGBM] [Info] Number of data points in the train set: 18850, number of used features: 21
[LightGBM] [Info] Start training from score 0.209443
 rmse score is 0.7830015625049729
